In [1]:
using Pkg
Pkg.activate("D:\\Science\\Code\\Julia\\envr")

  Activating project at `D:\Science\Code\Julia\envr`


In [1]:
using StaticArrays, DifferentialEquations, DynamicalSystems, BenchmarkTools

In [18]:
function TM(u, p, t)

    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    
    # u[1] - E; u[2] - x; u[3] - y
    # α - 1, τ - 2, τD - 3, τy - 4, J - 5, xthr - 6, ythr - 7, U0 - 8, ΔU0 - 9, β - 10, I0 -11

    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end;

In [19]:
t = 1000.0; tt = 250.0; tstep = 0.001
trange = range(0.0, t+tt, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [20]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.3;  const ΔU0 = 0.305; I0 = -1.40;

In [21]:
u0 = SA[17.6055, 0.6181, 0.429232]
p  = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
ds = ContinuousDynamicalSystem(TM, u0, p)

3-dimensional continuous dynamical system
 state:       [17.6055, 0.6181, 0.429232]
 rule f:      TM
 in-place?    false
 jacobian:    ForwardDiff
 parameters:  [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.3, 0.305, 0.3, -1.4]

In [8]:
@btime trajectory(ds, t, Δt = tstep; Ttr = tt, diffeq = integ_set);

  446.226 ms (82 allocations: 22.89 MiB)


In [9]:
@btime lyapunovspectrum(ds, t; Ttr = tt, diffeq = integ_set)

  996.751 ms (106 allocations: 8.53 KiB)


3-element Vector{Float64}:
  0.0004454263726357495
 -2.772583420776396
 -2.7695140699031464

In [30]:
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [31]:
@btime trajectory(ds, t, Δt = tstep; Ttr = tt, diffeq = integ_set);

  443.341 ms (82 allocations: 22.89 MiB)
